# Reliable Text-to-SQL Modeling on MIMIC III dataset without schema

This Jupyter notebook serves as a comprehensive guide to run Text-to-SQL model for Electronic Health Records (EHRs).

## Steps in This Jupyter Notebook
- [x] Step 1: Initial Setup
- [x] Step 2: Load and Prepare Datasets
- [x] Step 3: Construct a Text-to-SQL Model
- [x] Step 4: Model Evaluation

## Getting Started

Begin your journey with the EHRSQL task by following these structured steps (from Step 1 to Step 8). Each section is designed to guide you smoothly through the process, from setup to submission. We're eager to see the innovative solutions you'll bring to the field of Text-to-SQL modeling for electronic health records.

## Step 1: Initial Setup

As part of the intial setup we will pull code to Colab environment, install required Python dependencies and mount a GCP bucket to store intermediate results.

Clone the GitHub Repository created for this project.  It is a modified version of the original EHRSQL repo created by the writers of the paper.

In [ ]:
%cd /content
!rm -rf EHRSQL

%ls -al

# Cloning the GitHub repository
!git clone -q https://github.com/BizUnix/EHRSQL.git
%cd EHRSQL



/content
total 16
drwxr-xr-x 1 root root 4096 Apr  3 13:37 ./
drwxr-xr-x 1 root root 4096 Apr  4 19:35 ../
drwxr-xr-x 4 root root 4096 Apr  3 13:37 .config/
drwxr-xr-x 1 root root 4096 Apr  3 13:37 sample_data/
/content/EHRSQL


Install Required Python Packages:

In [ ]:
# Installing dependencies

!pip install -q transformers
!pip install -q sentencepiece
!pip install -q func_timeout


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Use the `%load_ext` magic command to automatically reload modules before executing a new line:

In [ ]:
%load_ext autoreload
%autoreload 2

Mount a GCP bucket for storing intermediate results and output

In [ ]:
# prompt: get gcsfuse

!apt-get update -qq
!apt-get install -qq -y fuse
!curl -LO https://github.com/googlecloudplatform/gcsfuse/releases/download/v1.0.1/gcsfuse_1.0.1_amd64.deb
!dpkg -i gcsfuse_1.0.1_amd64.deb




W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 5072k  100 5072k    0     0  5576k      0 --:--:-- --:--:-- --:--:-- 29.2M
Selecting previously unselected package gcsfuse.
(Reading database ... 126213 files and directories currently installed.)
Preparing to unpack gcsfuse_1.0.1_amd64.deb ...
Unpacking gcsfuse (1.0.1) ...
Setting up gcsfuse (1.0.1) ...


In [ ]:
from google.colab import auth;
auth.authenticate_user()

# Replace with your project ID and bucket name
!gcloud config set project striking-yen-455401-m9
!gcsfuse -v

!mkdir -p /content/cse6250_h1

# Mount bucket
!gcsfuse cse6250_h1 /content/cse6250_h1


Updated property [core/project].
gcsfuse version 1.0.1 (Go version go1.20.5)
I0404 19:37:24.807445 2025/04/04 19:37:24.807417 Start gcsfuse/1.0.1 (Go version go1.20.5) for app "" using mount point: /content/cse6250_h1


## Step 2: Load Data and Prepare Datasets

Now that we have our environment and paths set up, the next step is to load the data and prepare it for our model.  This involves preprocessing the MIMIC-III database and storing it into SQLLITE, reading the data from JSON files, splitting it into training and validation sets, and then initializing our dataset object.

### Preprocess MIMIC-III Data

Use a secure copy of the MIMIC-III data on Google Cloud Platform bucket to preprocess data and create the SQLLITE Mimic III dataset.


In [ ]:
%cd preprocess
!python preprocess_db.py --data_dir "/content/cse6250_h1/mimic-iii" --db_name "mimic_iii"
%cd ..

/content/ehrsql/preprocess
Processing PATIENTS, ADMISSIONS, ICUSTAYS, TRANSFERS
num_cur_patient: 0
num_non_cur_patient: 1000
num_patient: 1000
PATIENTS, ADMISSIONS, ICUSTAYS, TRANSFERS processed (took 15.0231 secs)
Processing D_ICD_DIAGNOSES, D_ICD_PROCEDURES, D_LABITEMS, D_ITEMS
D_ICD_DIAGNOSES, D_ICD_PROCEDURES, D_LABITEMS, D_ITEMS processed (took 2.0864 secs)
Processing DIAGNOSES_ICD table
DIAGNOSES_ICD processed (took 2.9386 secs)
Processing PROCEDURES_ICD table
PROCEDURES_ICD processed (took 2.2436 secs)
Processing LABEVENTS table
LABEVENTS processed (took 200.0603 secs)
Processing PRESCRIPTIONS table
PRESCRIPTIONS processed (took 55.0977 secs)
Processing COST table
COST processed (took 5.326 secs)
Processing CHARTEVENTS table
[########################################] | 100% Completed | 24m 45s
CHARTEVENTS processed (took 1496.1085 secs)
Processing INPUTEVENTS_CV table
INPUTEVENTS_CV processed (took 165.6402 secs)
Processing OUTPUTEVENTS table
OUTPUTEVENTS processed (took 35.9414

### Save SQLLite DB to GCP Bucket

This will help skip the preprocess step upon restarts


In [ ]:

# Move SQLLite to Google Bucket
!cp -r /content/ehrsql/dataset/ehrsql/mimic_iii/mimic_iii.sql* /content/cse6250_h1/


## Step 3: Construct a Text-to-SQL Baseline Model

In this step, we set up and train a T5 model to translate natural language queries into SQL statements. The process involves several key stages including argument parsing, model initialization, data preparation, and the actual training.


### Check GPU and connect to Google Drive (to store output)

In [ ]:
import torch
print(f"Torch Version: {torch.__version__}")

if torch.cuda.is_available():
    gpu_id = torch.cuda.current_device()
    print(f"GPU ID: {gpu_id}")
    print(f"GPU Name: {torch.cuda.get_device_name(gpu_id)}")
else:
    print("No GPU detected!")

Torch Version: 2.6.0+cu124
GPU ID: 0
GPU Name: Tesla T4


In [ ]:
# Check if the files /content/ehrsql/dataset/ehrsql/mimic_iii/mimic_iii.sql* exist
# if they don't download them from Google Cloud Platform bucket

!if [ ! -f "/content/cse6250_h1/mimic_iii.sql" ]; then
  cp -r /content/cse6250_h1/mimic_iii.sql* /content/cse6250_h1/

### Training the (T5) Model

Finally, we train the model on the dataset. The training process involves learning to generate SQL queries from textual descriptions through iterative forward and backward passes, loss computation, and parameter updates.  This version of the training use the pretrained T5 base model and trains it on the EHRSQL dataset without schema.

In [ ]:
!rm -rf outputs/ehrsql_mimic3_t5_base
!ls outputs

eval_ehrsql_mimic3_t5_base__mimic3_valid  eval_ehrsql_mimic3_t5_base_schema__mimic3_valid


In [ ]:
!python T5/main.py --config T5/config/ehrsql/training/ehrsql_mimic3_t5_base.yaml --CUDA_VISIBLE_DEVICES 0 --model_name t5-base



Current device: cuda:0
2025-04-01 01:33:51 | INFO | Namespace(exp_name='ehrsql_mimic3_t5_base', load_model_path=None, device='cuda', num_workers=50, random_seed=0, report_every_step=50, eval_batch_size=8, save_every_step=-1, save_every_epoch=False, show_eval_sample=True, eval_every_step=5000, eval_metric='loss', keep_last_ckpt=-1, early_stop_patience=-1, training_data_ratio=1.0, bf16=False, use_wandb=False, wandb_project=None, dataset='ehrsql', db_id='mimic_iii', train_data_path='dataset/ehrsql/mimic_iii/train.json', valid_data_path='dataset/ehrsql/mimic_iii/valid.json', output_dir='outputs', output_file='prediction_raw.json', model_name='t5-base', db_path=None, add_schema=False, add_column_type=False, shuffle_schema=False, tables_path=None, condition_value=True, warmup_steps=0, total_epoch=-1, total_step=100000, train_batch_size=4, accumulation_steps=8, lr=0.0001, scheduler_steps=None, optim='adam', scheduler='fixed', max_grad_norm=1.0, weight_decay=0.1, init_weights=False, num_beams=

### Store Model Output to GCP Bucket

In [ ]:
#copy model output to Google Cloud Bucket
!cp -r outputs/ehrsql_mimic3_t5_base /content/cse6250_h1/outputs


## Step 4: Model Evaluation

In this step, we will evaluate the model's performance across all queries, using the Reliability Score (RS) as our evaluation metric. This will provide a baseline understanding of the model's reliability scroe without filtering for unanswerable queries.

### Prepare for Running Inference

Load model from Google Drive to output directory, load SQL Database and get ready to run inference

In [ ]:
#if directory outputs/ehrsql_mimic3_t5_base doesn't exist, create it
!mkdir -p outputs/ehrsql_mimic3_t5_base

#copy model from GCP bucket to output directory
!cp -r /content/cse6250_h1/outputs/ehrsql_mimic3_t5_base/* outputs/ehrsql_mimic3_t5_base

#copy SQL lite database from Google cloud
!cp -r /content/cse6250_h1/mimic_iii.sql* dataset/ehrsql/mimic_iii/


In [ ]:
# List output
!ls outputs/ehrsql_mimic3_t5_base/*

# cleanup output directory for inferences
!rm -rf outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid
!rm -rf outputs/eval_ehrsql_mimic3_t5_base__mimic3_test

!ls -al outputs/ehrsql_mimic3_t5_base


outputs/ehrsql_mimic3_t5_base/checkpoint_best.pth.tar  outputs/ehrsql_mimic3_t5_base/train.log
total 2612708
drwxr-xr-x 2 root root       4096 Apr  4 19:38 .
drwxr-xr-x 4 root root       4096 Apr  4 20:41 ..
-rw-r--r-- 1 root root 2675193510 Apr  4 19:38 checkpoint_best.pth.tar
-rw-r--r-- 1 root root     201226 Apr  4 19:38 train.log


### Run inference with Validation Data

In [ ]:
# Run inferences for validation data
!python T5/main.py --config T5/config/ehrsql/eval/ehrsql_mimic3_t5_base__mimic3_valid.yaml --output_file prediction_raw.json --CUDA_VISIBLE_DEVICES 0


Current device: cuda:0
spiece.model: 100% 792k/792k [00:00<00:00, 16.1MB/s]
tokenizer.json: 100% 1.39M/1.39M [00:00<00:00, 15.9MB/s]
config.json: 100% 1.21k/1.21k [00:00<00:00, 11.0MB/s]
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
100% 1122/1122 [00:00<00:00, 449948.28it/s]
loaded 1122 test examples from dataset/ehrsql/mimic_iii/valid.json
2025-04-04 19:39:08.825476: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743795548.845436 

### Run Inference with Test Data

In [ ]:
# Run inferences for test data
!python T5/main.py --config T5/config/ehrsql/eval/ehrsql_mimic3_t5_base__mimic3_test.yaml --output_file prediction_raw.json --CUDA_VISIBLE_DEVICES 0


Current device: cuda:0
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
100% 1786/1786 [00:00<00:00, 534996.93it/s]
loaded 1786 test examples from dataset/ehrsql/mimic_iii/test.json
2025-04-04 20:41:32.411255: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743799292.438276   21907 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:17437992

### Run abstention with or without threshold to generate final files

We save these predictions to a JSON file in a designated results directory, creating the directory if necessary.  The JSON files are then moved to GCP to allow for generating performance metrics.

In [ ]:
# Run abstention wihtout threshold for validation data
!python T5/abstain_with_entropy.py --inference_result_path outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid --input_file prediction_raw.json --output_file prediction.no_threshold.json --threshold -1

# Run abstention with threshold of 0.14923561
!python T5/abstain_with_entropy.py --inference_result_path outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid --input_file prediction_raw.json --output_file prediction.fixed_threshold.json --threshold 0.14923561

# Backup all predictions to GCP buucket
!cp -r outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid /content/cse6250_h1/outputs


/content/EHRSQL/T5/abstain_with_entropy.py:18: UserWarning: Threshold value is not set! All predictions are sent to the database.
  warnings.warn("Threshold value is not set! All predictions are sent to the database.")


In [ ]:
# Run abstention wihtout threshold for test data
!python T5/abstain_with_entropy.py --inference_result_path outputs/eval_ehrsql_mimic3_t5_base__mimic3_test --input_file prediction_raw.json --output_file prediction.no_threshold.json --threshold -1

# Run abstention with threshold of 0.14923561
!python T5/abstain_with_entropy.py --inference_result_path outputs/eval_ehrsql_mimic3_t5_base__mimic3_test --input_file prediction_raw.json --output_file prediction.fixed_threshold.json --threshold 0.14923561

# Backup all predictions to GCP buucket
!cp -r outputs/eval_ehrsql_mimic3_t5_base__mimic3_test /content/cse6250_h1/outputs


/content/EHRSQL/T5/abstain_with_entropy.py:18: UserWarning: Threshold value is not set! All predictions are sent to the database.
  warnings.warn("Threshold value is not set! All predictions are sent to the database.")


### Evaluate Performance


#### With Validation Data

In this step we evaluate performance of the results produced in prior steps with the JSON produced for Validation Data

In [ ]:
# Check if the output already exists on
# outputs/eval_ehrsql_mimic3_t5_base__mimic3_test if not copy it from GCP- this
# step allows for repeatable runs without running inference

!if [ ! -f "outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid" ]; then cp -r /content/cse6250_h1/outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid/ outputs/; fi

!echo "Key Metrics for Validation Data - without Scehma"

!echo "With No Threshold"
!python evaluate.py --db_path ./dataset/ehrsql/mimic_iii/mimic_iii.sqlite --data_file dataset/ehrsql/mimic_iii/valid.json --pred_file ./outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid/prediction.no_threshold.json

!echo "With Fixed Threshold"
!python evaluate.py --db_path ./dataset/ehrsql/mimic_iii/mimic_iii.sqlite --data_file dataset/ehrsql/mimic_iii/valid.json --pred_file ./outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid/prediction.fixed_threshold.json



Key Metrics for Validation Data - without Scehma
With No Threshold
{
  "precision_ans": 67.74,
  "recall_ans": 100.0,
  "f1_ans": 80.77,
  "precision_exec": 65.51,
  "recall_exec": 96.71,
  "f1_exec": 78.11
}
With Fixed Threshold
{
  "precision_ans": 94.5,
  "recall_ans": 94.87,
  "f1_ans": 94.68,
  "precision_exec": 93.58,
  "recall_exec": 93.95,
  "f1_exec": 93.76
}


#### With Test Data

In this step we evaluate performance of the results produced in prior steps with the JSON produced for Test Data

In [ ]:
# Check if the output already exists on
# outputs/eval_ehrsql_mimic3_t5_base__mimic3_test if not copy it from GCP- this
# step allows for repeatable runs without running inference

!if [ ! -f "outputs/eval_ehrsql_mimic3_t5_base__mimic3_test" ]; then cp -r /content/cse6250_h1/outputs/eval_ehrsql_mimic3_t5_base__mimic3_test/ outputs/; fi

!echo "Key Metrics for Test Data - without Scehma"

!echo "With No Threshold"
!python evaluate.py --db_path ./dataset/ehrsql/mimic_iii/mimic_iii.sqlite --data_file dataset/ehrsql/mimic_iii/test.json --pred_file ./outputs/eval_ehrsql_mimic3_t5_base__mimic3_test/prediction.no_threshold.json

!echo "With Fixed Threshold"
!python evaluate.py --db_path ./dataset/ehrsql/mimic_iii/mimic_iii.sqlite --data_file dataset/ehrsql/mimic_iii/test.json --pred_file ./outputs/eval_ehrsql_mimic3_t5_base__mimic3_test/prediction.fixed_threshold.json



Key Metrics for Test Data - without Scehma
With No Threshold
{
  "precision_ans": 67.08,
  "recall_ans": 100.0,
  "f1_ans": 80.29,
  "precision_exec": 65.12,
  "recall_exec": 97.08,
  "f1_exec": 77.95
}
With Fixed Threshold
{
  "precision_ans": 90.37,
  "recall_ans": 93.99,
  "f1_ans": 92.14,
  "precision_exec": 89.57,
  "recall_exec": 93.16,
  "f1_exec": 91.33
}
